## Defining the Mini DCM-like System

We will build a 3-Node causal system that:
 * Evolves over time
 * Receives an external input (Stimulus)
 * Has a connectivity structure (Directed graph)
 * Simple enough to use basic linear algebra

#### States, input, observations

- $x_t \in \mathbb{R}^3$: hidden states at time $t$  
  - $x_{t,1}$: activity of node 1  
  - $x_{t,2}$: activity of node 2  
  - $x_{t,3}$: activity of node 3  

- $u_t \in \mathbb{R}$: scalar external input at time $t$  
  This can switched on and off to see stimulus effect.  

- $y_t \in \mathbb{R}^3$: observed data at time $t$  
  Noisy measurements of each node (That is what we collect mostly from neuroimaging methods)  

So we have a time series:

$$
\{(u_t, y_t)\}_{t=0}^{T-1}
$$


#### Dynamics

This is the model of how hidden states evolve over time.

Evolution equation:
$$
x_{t+1} = A x_t + C u_t + w_t
$$

Where:

- $A$ is a $3 \times 3$ matrix: **effective connectivity**  
  - element $A_{ij}$: how much node $j$ at time $t$ influences node $i$ at time $t+1$

- $C$ is a $3 \times 1$ vector: how strongly the **input** drives each node

- $w_t \sim \mathcal{N}(0,\sigma_w^2 I_3)$: **noise**  
  - random fluctuations in the hidden states

To interpret:

- The next state of each node is a linear combination of  
  - the current states of all nodes  
  - the current input  
  - random noise  

This is the “toy DCM”: the matrix $A$ encodes directed causal influences.

#### Observation

We don't observe $x_t$ directly, we see it's noisy versions

$$
y_t=x_t + v_t
$$

Where $v_t \sim \mathcal{N}(0,\sigma_w^2 I_3)$ is observation noise.

So the measurement is the latent state plus noise, with no extra transformation. This is deliberately minimal:

- no hemodynamic model

- no nonlinearities

- just linear + Gaussian


#### Matrices and causal graph

The graph structure is encoded in **which entries of \(A\) are allowed to be non zero**.

- If $(A_{21} \ne 0)$, then node 1 → node 2  
- If $(A_{21} = 0)$, then node 1 does not directly drive node 2  

Self-connections
- $(A_{11}, A_{22}, A_{33})$: self-dynamics

External input:

- $(C_1 \ne 0)$: input drives node 1  
- $(C_2 \ne 0)$: input drives node 2  
- $(C_3 \ne 0)$: input drives node 3  

So:

- A **model** is:  
  - a pattern of zeros / non-zeros in \(A\) and \(C\)  
  - plus the actual parameter values for the non-zero entries

Later, **Bayesian model comparison** will compare **patterns** (who connects to whom), not just parameter values.

### "True" Model Choice

1. **Graph structure:**
   - $(1 \rightarrow 2 \rightarrow 3)$
   - input drives node 1 only

2. **Numeric example:**

$$
[A =
\begin{bmatrix}
0.90 & 0.00 & 0.00 \\
0.30 & 0.80 & 0.00 \\
0.00 & 0.25 & 0.85
\end{bmatrix},
\qquad
C =
\begin{bmatrix}
1.0 \\
0.0 \\
0.0
\end{bmatrix}]
$$

**Meanings:**

- **Node 1:**
  - strong self-connection \(0.90\)
  - no input from nodes 2 or 3

- **Node 2:**
  - self-connection \(0.80\)
  - receives from node 1 with weight \(0.30\)

- **Node 3:**
  - self-connection \(0.85\)
  - receives from node 2 with weight \(0.25\)

  -> Self-connections could have been self inhibitions too.

- **Input:**
  - directly adds to node 1’s state

**Noise:**

- $(\sigma_w = 0.05,\ \sigma_v = 0.05)$


This fixed pair $(A_{\text{true}}, C_{\text{true}})$ is our **ground truth toy brain**.

Everything else in the project is built around:

- simulating from this model  
- proposing alternative models with different zero patterns  
- fitting them  
- comparing them  